In [ ]:
from keras.applications import VGG16, InceptionV3, ResNet50V2, InceptionResNetV2, Xception, VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import os
import tensorflow as tfS
from keras import backend as K
import dill
#from convnetskeras.convnets import convnet

##Pastas
train_dir= 'C:\\Users\\bmfs9\\Downloads\\TESE\\Dataset_patches_FL2\\wra\\train'
val_dir= 'C:\\Users\\bmfs9\\Downloads\\TESE\\Dataset_patches_FL2\\wra\\validation'
test_dir= 'C:\\Users\\bmfs9\\Downloads\\TESE\\Dataset_patches\\wra\\test'

#FOCAL LOSS
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

    return binary_focal_loss_fixed

###############################################
#Modelo
################################

conv=VGG19(weights='imagenet', include_top=False, input_shape= (256,256,3))


model = conv.output
model = layers.Flatten()(model)
model = layers.Dense(1024, activation='relu')(model)
model = layers.BatchNormalization()(model)
model = layers.Dense(512, activation='relu')(model)
model = layers.BatchNormalization()(model)
model = layers.Dense(256, activation='relu')(model)
model = layers.BatchNormalization()(model)
model = layers.Dense(128, activation='relu')(model)
model = layers.BatchNormalization()(model)
model = layers.Dense(64, activation='relu')(model)
model = layers.BatchNormalization()(model)
model = layers.Dense(32, activation='relu')(model)
model = layers.BatchNormalization()(model)
out = layers.Dense(1, activation='sigmoid')(model)

model = models.Model(inputs=conv.input, outputs=out)


conv.trainable = True
set_trainable = False

for layer in model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

################################################
#TREINAR MODELO
######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255)

'''
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
'''
    


datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(  
    train_dir,
    target_size=(256,256),
    batch_size=20,
    class_mode='binary')

validation_generator = datagen.flow_from_directory(  
    val_dir,
    target_size=(256,256),
    batch_size=20,
    class_mode='binary')



#os.chdir('C:\\Users\\bmfs9\\Downloads\\TESE\\unw\\models\\teste')
callb= [keras.callbacks.ModelCheckpoint(filepath='C:\\Users\\bmfs9\\Downloads\\TESE\\unw\\models\\teste\\FL2_vgg19_wra.h5', 
                                        monitor='val_loss', 
                                        save_best_only=True)]


model.compile(loss=binary_focal_loss(gamma=2., alpha=.25),
              optimizer=optimizers.Adam(lr=0.00001),
              metrics=['accuracy']
)

history = model.fit(
    train_generator,
    #steps_per_epoch=260,
    epochs=60,
    validation_data=validation_generator,
    #validation_steps=70,
    callbacks=callb)




#criar plots
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(21, len(acc) + 21) #change to 1
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

dict_h=history.history

###########################
#Teste
############################

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=20,
    class_mode='binary')


model=load_model('C:\\Users\\bmfs9\\Downloads\\TESE\\unw\\models\\teste\\preliminary_tests\\vgg19.h5'#, 
                 #custom_objects={'binary_focal_loss_fixed': binary_focal_loss()} 
                 )

teste= model.evaluate(test_generator)